<a href="https://colab.research.google.com/github/akira-harada/SPECIAL_BLEND/blob/main/SPECIAL_BLEND_pyinterface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/akira-harada/SPECIAL_BLEND.git

In [ ]:
import sys

In [ ]:
!pip install ninja
!pip install meson

In [ ]:
!{sys.executable} -m numpy.f2py --quiet -c /content/SPECIAL_BLEND/SPECIAL_BLEND.f90 -m SPECIAL_BLEND

In [ ]:
#Skip this cell when you use your own observational data.
%run /content/SPECIAL_BLEND/event_generator.py

In [ ]:
#@title
# define functions
%config InlineBackend.figure_format = 'retina'
import numpy as np
import csv
import matplotlib.pyplot as plt
import SPECIAL_BLEND as SB

def main():
  params = np.loadtxt('/content/SPECIAL_BLEND/parameters.dat')
  # 'parameters.dat' file has the following contents:
  # assumed gbeta,
  # distance to the SN [kpc],
  # detector mass [kton],
  # parameter search grid number,
  # minimum search mass [Msun],
  # maximum search mass [Msun],
  # minimum search radius [km],
  # maximum search radius [km],
  # minimum search energy [erg],
  # and maximum search energy [erg]
  origdata = np.loadtxt('/content/time_energy.dat')
  # 'time_energy.dat' file has the time and energy of each event: first column is time, second column is energy
  analysis_mode = int(params[10])# 1:unbinned, 2:binned, 3:Gaussian approximation
  tmin = params[13]
  tmax = params[14]
  data = loaddata(tmin,tmax,origdata)
  if analysis_mode == 1:
    print("unbinned analysis")
    mlogLH,mass,rad,et = unbinned_likelihood(data,params)
    print("likelihood calculation completed")
  elif analysis_mode == 2:
    print("binned analysis")
    mlogLH,mass,rad,et = binned_likelihood(data,params)
    print("likelihood calculation completed")
  elif analysis_mode == 3:
    print("Gaussian-approximation analysis")
    mlogLH,mass,rad,et = GA_likelihood(data,params)
    print("likelihood calculation completed")
  else:
    print("invalid analysis mode")
  return mass,rad,et,mlogLH


def loaddata(tmin,tmax,origdata):
  cuttdata = origdata[:,0].clip(tmin,tmax)
  diftdata = cuttdata - origdata[:,0]
  islice = np.where(diftdata==0.0)[0].min()
  fslice = np.where(diftdata==0.0)[0].max()+1
  data = origdata[islice:fslice,:]
  origsize = origdata[:,0].shape[0]
  datsize = data.shape[0]
  print("used event number {:d} /total event number {:d}".format(datsize,origsize))
  print("{:d} events are outside [tmin,tmax] and neglected".format(origsize-datsize))
  return data


def unbinned_likelihood(data,params):
  datsize = data.shape[0]
  tdata = data[:,0]
  edata = data[:,1]
  gbeta = params[0]
  dist = params[1]
  Mdet = params[2]
  nparam = int(params[3])
  mmin = params[4]
  mmax = params[5]
  rmin = params[6]
  rmax = params[7]
  emin = params[8]
  emax = params[9]

  mass = np.linspace(mmin,mmax,nparam)
  rad  = np.linspace(rmin,rmax,nparam)
  et   = np.linspace(emin,emax,nparam)

  mlogLH = SB.eval_unbinned_likelihood(tdata,edata,mass,rad,et,gbeta,dist,Mdet)

  return mlogLH,mass,rad,et


def binned_likelihood(data,params):
  datsize = data.shape[0]
  tdata = data[:,0]
  edata = data[:,1]
  gbeta = params[0]
  dist = params[1]
  Mdet = params[2]
  nparam = int(params[3])
  mmin = params[4]
  mmax = params[5]
  rmin = params[6]
  rmax = params[7]
  emin = params[8]
  emax = params[9]
  tbinnumber = int(params[11])
  dt_ini = params[12]
  tmin   = params[13]
  tmax   = params[14]

  t_bin,dt,e_bin,de,hist,status = SB.binning(tdata,edata,dt_ini,tmin,tmax,tbinnumber)
  if status >= 1:
    print("binning error, try another bin number")
    interrupt_by_undefined_function()

  mass = np.linspace(mmin,mmax,nparam)
  rad  = np.linspace(rmin,rmax,nparam)
  et   = np.linspace(emin,emax,nparam)

  mlogLH = SB.eval_binned_likelihood(t_bin,dt,e_bin,de,hist,mass,rad,et,gbeta,dist,Mdet)
  return mlogLH,mass,rad,et

def GA_likelihood(data,params):
  datsize = data.shape[0]
  tdata = data[:,0]
  edata = data[:,1]
  gbeta = params[0]
  dist = params[1]
  Mdet = params[2]
  nparam = int(params[3])
  mmin = params[4]
  mmax = params[5]
  rmin = params[6]
  rmax = params[7]
  emin = params[8]
  emax = params[9]
  tbinnumber = int(params[11])
  dt_ini = params[12]
  tmin   = params[13]
  tmax   = params[14]

  t_bin,dt,thist,status = SB.time_binning(tdata,dt_ini,tmin,tmax,tbinnumber)
  if status >= 1:
    print("time binning error, try another bin number")
    interrupt_by_undefined_function()
  e_ave,status = SB.event_energy_averaging(thist,edata)
  if status >= 1:
    print("{:d}-th time bin has no events".format(status))
    print("try another time bin number")
    interrupt_by_undefined_function()

  mass = np.linspace(mmin,mmax,nparam)
  rad  = np.linspace(rmin,rmax,nparam)
  et   = np.linspace(emin,emax,nparam)

  mlogLH = SB.eval_gaussian_likelihood(t_bin,dt,thist,e_ave,mass,rad,et,gbeta,dist,Mdet)
  return mlogLH,mass,rad,et

def marginalizeLH(mass,rad,et,mlogLH):
  LH_MR,CI95level,CI68level,peak = SB.mr_marginalize(mass,rad,et,mlogLH)
  MRvals  = [CI95level,CI68level,peak[0],peak[1]]
  LH_RE,CI95level,CI68level,peak = SB.re_marginalize(mass,rad,et,mlogLH)
  REvals  = [CI95level,CI68level,peak[0],peak[1]]
  LH_EM,CI95level,CI68level,peak = SB.em_marginalize(mass,rad,et,mlogLH)
  EMvals  = [CI95level,CI68level,peak[0],peak[1]]
  LH_M,CIBFM  = SB.m_marginalize(mass,rad,et,mlogLH)
  LH_R,CIBFR  = SB.r_marginalize(mass,rad,et,mlogLH)
  LH_E,CIBFE  = SB.e_marginalize(mass,rad,et,mlogLH)
  print("1D marginalized result")
  print("mass = {:e} +{:.2e}/-{:.2e} (68%) +{:.2e}/-{:.2e} (95%)".format(CIBFM[2],CIBFM[3]-CIBFM[2],CIBFM[2]-CIBFM[1],CIBFM[4]-CIBFM[2],CIBFM[2]-CIBFM[0]))
  if (CIBFM[0] > CIBFM[1]) or (CIBFM[1] > CIBFM[2]) or (CIBFM[2] > CIBFM[3]) or (CIBFM[3] > CIBFM[4]):
    print('\033[35m'+"1D CI estimation error in function marginalizeLH()"+'\033[0m')
    print('\033[35m'+"array CIBFM containes [lower edge of 95%-CI, lower edge of 68%-CI, best fit, upper edge of 68%-CI, upper edge of 95%-CI]"+'\033[0m')
    print('\033[35m'+"but ascending nature of CIBFM does not hold"+'\033[0m')
    print('\033[35m'+"retry with another parameter search range"+'\033[0m')
    print('\033[35m'+"or data quality is too low to analyze"+'\033[0m')
  print("radius = {:e} +{:.2e}/-{:.2e} (68%) +{:.2e}/-{:.2e} (95%)".format(CIBFR[2],CIBFR[3]-CIBFR[2],CIBFR[2]-CIBFR[1],CIBFR[4]-CIBFR[2],CIBFR[2]-CIBFR[0]))
  if (CIBFR[0] > CIBFR[1]) or (CIBFR[1] > CIBFR[2]) or (CIBFR[2] > CIBFR[3]) or (CIBFR[3] > CIBFR[4]):
    print('\033[35m'+"1D CI estimation error in function marginalizeLH()"+'\033[0m')
    print('\033[35m'+"array CIBFR containes [lower edge of 95%-CI, lower edge of 68%-CI, best fit, upper edge of 68%-CI, upper edge of 95%-CI]"+'\033[0m')
    print('\033[35m'+"but ascending nature of CIBFR does not hold"+'\033[0m')
    print('\033[35m'+"retry with another parameter search range"+'\033[0m')
    print('\033[35m'+"or data quality is too low to analyze"+'\033[0m')
  print("energy = {:e} +{:.2e}/-{:.2e} (68%) +{:.2e}/-{:.2e} (95%)".format(CIBFE[2],CIBFE[3]-CIBFE[2],CIBFE[2]-CIBFE[1],CIBFE[4]-CIBFE[2],CIBFE[2]-CIBFE[0]))
  if (CIBFE[0] > CIBFE[1]) or (CIBFE[1] > CIBFE[2]) or (CIBFE[2] > CIBFE[3]) or (CIBFE[3] > CIBFE[4]):
    print('\033[35m'+"1D CI estimation error in function marginalizeLH()"+'\033[0m')
    print('\033[35m'+"array CIBFE containes [lower edge of 95%-CI, lower edge of 68%-CI, best fit, upper edge of 68%-CI, upper edge of 95%-CI]"+'\033[0m')
    print('\033[35m'+"but ascending nature of CIBFR does not hold"+'\033[0m')
    print('\033[35m'+"retry with another parameter search range"+'\033[0m')
    print('\033[35m'+"or data quality is too low to analyze"+'\033[0m')
  print()
  print("2D marginalized result")
  print("M-R: the best fit is (M,R)=({:.2e},{:.2e}), and the levels of CI is {:.2e} (68%) and {:.2e} (95%)".format(MRvals[2],MRvals[3],MRvals[1],MRvals[0]))
  if (LH_MR.max() < MRvals[0]) or (LH_MR.max() < MRvals[1]) or (MRvals[0] > MRvals[1]):
    print('\033[35m'+"2D CI estimation error in function marginalizeLH()"+'\033[0m')
    if (LH_MR.max() < MRvals[0]) or (LH_MR.max() < MRvals[1]):
      print('\033[35m'+"levels of CI exceeds maximum value of the likelihood"+'\033[0m')
    if (MRvals[0] > MRvals[1]):
      print('\033[35m'+"95%-CI contour may be inside 68%-CI contour"+'\033[0m')
    print('\033[35m'+"retry with another parameter search range"+'\033[0m')
    print('\033[35m'+"or data quality is too low to analyze"+'\033[0m')
  print("R-E: the best fit is (R,E)=({:.2e},{:.2e}), and the levels of CI is {:.2e} (68%) and {:.2e} (95%)".format(REvals[2],REvals[3],REvals[1],REvals[0]))
  if (LH_RE.max() < REvals[0]) or (LH_RE.max() < REvals[1]) or (REvals[0] > REvals[1]):
    print('\033[35m'+"2D CI estimation error in function marginalizeLH()"+'\033[0m')
    if (LH_RE.max() < REvals[0]) or (LH_RE.max() < REvals[1]):
      print('\033[35m'+"levels of CI exceeds maximum value of the likelihood"+'\033[0m')
    if (REvals[0] > REvals[1]):
      print('\033[35m'+"95%-CI contour may be inside 68%-CI contour"+'\033[0m')
    print('\033[35m'+"retry with another parameter search range"+'\033[0m')
    print('\033[35m'+"or data quality is too low to analyze"+'\033[0m')
  print("E-M: the best fit is (E,M)=({:.2e},{:.2e}), and the levels of CI is {:.2e} (68%) and {:.2e} (95%)".format(EMvals[2],EMvals[3],EMvals[1],EMvals[0]))
  if (LH_EM.max() < EMvals[0]) or (LH_EM.max() < EMvals[1]) or (EMvals[0] > EMvals[1]):
    print('\033[35m'+"2D CI estimation error in function marginalizeLH()"+'\033[0m')
    if (LH_EM.max() < EMvals[0]) or (LH_EM.max() < EMvals[1]):
      print('\033[35m'+"levels of CI exceeds maximum value of the likelihood"+'\033[0m')
    if (EMvals[0] > EMvals[1]):
      print('\033[35m'+"95%-CI contour may be inside 68%-CI contour"+'\033[0m')
    print('\033[35m'+"retry with another parameter search range"+'\033[0m')
    print('\033[35m'+"or data quality is too low to analyze"+'\033[0m')
  save_likelihood(mass,rad,et,mlogLH,LH_MR,LH_RE,LH_EM,LH_M,LH_R,LH_E)

  return LH_MR,MRvals,LH_RE,REvals,LH_EM,EMvals,LH_M,LH_R,LH_E


def save_likelihood(mass,rad,et,mlogLH,LH_MR,LH_RE,LH_EM,LH_M,LH_R,LH_E):
  Marray = np.stack([mass,LH_M],1)
  Mhead = [['mass',  'LH_M']]
  with open("LH_M.csv","w",newline="") as f:
    writer = csv.writer(f)
    writer.writerows(np.concatenate([Mhead,Marray],0))

  Rarray = np.stack([rad, LH_R],1)
  Rhead = [['radius','LH_R']]
  with open("LH_R.csv","w",newline="") as f:
    writer = csv.writer(f)
    writer.writerows(np.concatenate([Mhead,Marray],0))

  Earray = np.stack([et,  LH_E],1)
  Ehead = [['energy','LH_E']]
  with open("LH_E.csv","w",newline="") as f:
    writer = csv.writer(f)
    writer.writerows(np.concatenate([Mhead,Marray],0))

  mrm,mrr = np.meshgrid(mass,rad)
  rer,ree = np.meshgrid(rad,et)
  eme,emm = np.meshgrid(et,mass)

  mrm1    = np.ravel(mrm)
  mrr1    = np.ravel(mrr)
  LH_MR1  = np.ravel(LH_MR)
  MRarray = np.stack([mrm1,mrr1,LH_MR1]).T
  MRhead  = [['mass','radius','LH_MR']]
  with open("LH_MR.csv","w",newline="") as f:
    writer = csv.writer(f)
    writer.writerows(np.concatenate([MRhead,MRarray],0))

  rer1    = np.ravel(rer)
  ree1    = np.ravel(ree)
  LH_RE1  = np.ravel(LH_RE)
  REarray = np.stack([rer1,ree1,LH_RE1]).T
  REhead  = [['radius','energy','LH_RE']]
  with open("LH_RE.csv","w",newline="") as f:
    writer = csv.writer(f)
    writer.writerows(np.concatenate([REhead,REarray],0))

  eme1    = np.ravel(eme)
  emm1    = np.ravel(emm)
  LH_EM1  = np.ravel(LH_EM)
  EMarray = np.stack([eme1,emm1,LH_EM1]).T
  EMhead  = [['energy','mass','LH_EM']]
  with open("LH_EM.csv","w",newline="") as f:
    writer = csv.writer(f)
    writer.writerows(np.concatenate([EMhead,EMarray],0))


def visualize(mass,rad,et,mlogLH,LH_MR,MRvals,LH_RE,REvals,LH_EM,EMvals,LH_M,LH_R,LH_E):
  mrm,mrr = np.meshgrid(mass,rad)
  plt.contourf(mrm,mrr,LH_MR.T)
  plt.contour(mrm,mrr,LH_MR.T,colors=['white'],levels=[MRvals[0],MRvals[1]])
  plt.plot(MRvals[2],MRvals[3],color='white',marker='.',markersize=10)
  plt.xlabel(r"mass [$M_\odot$]")
  plt.ylabel(r"radius [km]")
  plt.show()

  rer,ree = np.meshgrid(rad,et)
  plt.contourf(rer,ree,LH_RE.T)
  plt.contour(rer,ree,LH_RE.T,colors=['white'],levels=[REvals[0],REvals[1]])
  plt.plot(REvals[2],REvals[3],color='white',marker='.',markersize=10)
  plt.xlabel(r"radius [km]")
  plt.ylabel(r"energy [erg]")
  plt.show()

  eme,emm = np.meshgrid(et,mass)
  plt.contourf(eme,emm,LH_EM.T)
  plt.contour(eme,emm,LH_EM.T,colors=['white'],levels=[EMvals[0],EMvals[1]])
  plt.plot(EMvals[2],EMvals[3],color='white',marker='.',markersize=10)
  plt.xlabel(r"energy [erg]")
  plt.ylabel(r"mass [$M_\odot$]")
  plt.show()

  plt.plot(mass,LH_M)
  plt.xlabel(r"mass [$M_\odot$]")
  plt.show()
  print()

  plt.plot(rad,LH_R)
  plt.xlabel(r"radius [km]")
  plt.show()
  print()

  plt.plot(et,LH_E)
  plt.xlabel(r"energy [erg]")
  plt.show()


In [ ]:
# calculate likelihood (mlogLH = minus log likelihood)
%time mass,rad,et,mlogLH=main()

In [ ]:
# marginalize likelihood
LH_MR,MRvals,LH_RE,REvals,LH_EM,EMvals,LH_M,LH_R,LH_E = marginalizeLH(mass,rad,et,mlogLH)

In [ ]:
# visualize likelihood
visualize(mass,rad,et,mlogLH,LH_MR,MRvals,LH_RE,REvals,LH_EM,EMvals,LH_M,LH_R,LH_E)